# Sustainable Agriculture - Cover Crops
_____

The purpose of this notebook is to demonstrate how the Descartes Labs platform can assist in quickly developing sustainable agriculture analysis and workflows. 

The notebook shows a cover crop prediction model, based on the methodology implemented in the paper ["Remote Sensing to Monitor Cover Crop Adoption in Southeastern Pennsylvania"](https://www.jswconline.org/content/jswc/70/6/340.full.pdf). The algorithm uses NDVI thresholds based on wintertime imagery following summertime row crops. NDVI thresholds are used to determine what pixels contain different amounts of cover crops (which can also be interpreted as wintertime vegetation). 

You can run the cells in this notebook one at a time by using `Shift-Enter`

In [1]:
# keep logging quiet
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.captureWarnings(True)

In [2]:
# import packages
import descarteslabs as dl
import descarteslabs.workflows as wf

import ipywidgets
from ipywidgets import Layout

import ipyleaflet
from ipyleaflet import GeoData

from shapely.geometry import Polygon

from collections import defaultdict
import geopandas as gpd

from cover_crops_wf import CoverCrops

### Set up an interactive map

In [3]:
cc_m = CoverCrops(wf.map)
cc_m.m.center = (40.1109, -76.6141)
cc_m.m.zoom = 13

### Display relevant Pennsylvania counties

The paper that motivates this work conducted a survey in the following PA counties: Berks, Lancaster, Lebanon, York. Here, we use the DL Places API to get the outlines of these counties and display them on an interactive map.

In [4]:
# Get counties of interest
places = dl.Places()
pa_counties = places.prefix('north-america_united-states_pennsylvania')['features']
pa_counties_oi = [p for p in pa_counties if p['properties']['name'] in ['Berks', 'Lancaster', 'Lebanon', 'York']]

geoms = []
names = []
for c in pa_counties_oi:
    geoms.append(Polygon(c['geometry']['coordinates'][0]))
    names.append(c['properties']['name'])

d = {'county': names, 'geometry': geoms}
gdf = gpd.GeoDataFrame(d, crs="EPSG:4326")
gdf.head()

# Add counties of interest to the map
# Add layer control
layer_control = ipyleaflet.LayersControl(position="topright")
cc_m.m.add_control(layer_control)

# Set up widget for metadata display
output = ipywidgets.Output(
    layout={'min_width':'150px','min_height':'20px',
            'max_width':'500px','max_height':'200px'}
)
output = wf.interactive.clearable.ClearableOutput(output)
output = output.children[0]
output_ctrl = ipyleaflet.WidgetControl(widget=output, position='bottomright')
cc_m.m.add_control(output_ctrl)
@output.capture()

# Set up hovering feature
def circle_hover(feature, **kwargs):
    output.clear_output()
    print(f"{feature['properties']['county']} County")
    
with output:
    # Add metadata to the map from a geodataframe
    geodata = gdf.__geo_interface__
    circle_marker = ipyleaflet.GeoJSON(
        data=geodata,
        hover_style={"fillColor": "#2E99DF", "fillOpacity": 0.0},
        style={"fillColor": "#2E99DF", "fillOpacity": 0.0},
        name="PA Counties of Interest"
    )
    circle_marker.on_hover(circle_hover)
    cc_m.m.add_layer(circle_marker)

### Allow for analysis flexibility with user-based year selection

We create a year selection box and add it to the interactive map. When the year is changed by a user, analysis is automatically re-run for the selected year.

### Display the interactive map

In [5]:
cc_m.m


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart the kernel and refresh the webpage.


In [6]:
cc_m.fig_output

Output()

### Define imagery layers for cover crop analysis
The below cell computes all of the layers needed to compute the cover crop analysis. 

First we define a Sentinel-2 image collection that is masked with an internal dlcloud product. A NDVI image collection is computed from the original image collection's red and nir bands, and two output layers are computed: (1) a maximum NDVI composite, and (2) a RGB composite, indexed by argmax NDVI.

The second layer is derived from the Cropland Data Layer (CDL). For this derived layer, we create a mask from all pixels labeled as corn or soy in the previous growing season. Cover crops are most likely to be grown following a season of corn or soy, so we filter the outputs to compute on only these regions.

In [7]:
# Control visualization with year parameter
year = wf.parameter("year", wf.Int)

# Get cloud masked Sentinel-2 data

# Define Image Collection
ic = wf.ImageCollection.from_id(
    "sentinel-2:L1C",
    wf.Datetime(year, month=2, day=23),
    wf.Datetime(year, month=4, day=1),
    processing_level="surface"
).pick_bands("red green blue nir")

# Define cloud masks image collection
cloudmask = (
    wf.ImageCollection.from_id(
        "sentinel-2:L1C:dlcloud:v1",
        wf.Datetime(year, month=2, day=23),
        wf.Datetime(year, month=4, day=1)
    ).pick_bands("valid_cloudfree") == 0
)

# Make an ImageCollectionGroupby object, for quicker
# lookups from `ic` by date (you can use it like a dict)
ic_date_groupby = ic.groupby(
    dates=("year", "month", "day")
)

# For each cloudmask date, pick the corresponding image from `ic` 
# by date, mosaic both, and mask them. (It may be that not all 
# scenes have cloudmasks processed, so this ensures we only 
# return scenes that do.)
masked_ic = cloudmask.groupby(
    dates=("year", "month", "day")
).map(
    lambda ymd, mask_imgs: ic_date_groupby[ymd].mosaic()
    .mask(mask_imgs.mosaic())
)

# Get max NDVI and visualize
nir, red = masked_ic.unpack_bands("nir red")
ndvi = wf.normalized_difference(nir, red).rename_bands('ndvi')
# ndvi = (nir - red)/(nir + red)
    
s2_max_ndvi = ndvi.max(axis='images')

# Get RGB based on max NDVI and visualize
s2_rgb = masked_ic.pick_bands("red green blue").sortby_composite(ndvi, operation="argmax")

# Visualize composites
s2_rgb.visualize('Sentinel-2 RGB, sorted by Max NDVI', scales=[[0, 0.3], [0, 0.3], [0, 0.3]], year=cc_m.year_select)
s2_max_ndvi.visualize('Sentinel-2 max NDVI', scales=[1.0, 0], colormap='Greys', year=cc_m.year_select)

# Get imagery from previous year's CDL
cdl = wf.ImageCollection.from_id(
    'usda:cdl:v1',
    start_datetime=wf.Datetime(year - 1),
    end_datetime=wf.Datetime(year)
).mosaic()

# Get crops of interest
combined = (cdl == 1) | (cdl == 5)

In [8]:
# Display combined crop mask
combined.mask(~combined).visualize(
    'CDL Corn and Soy',
    colormap='Wistia',
    checkerboard=False,
    year=cc_m.year_select)

### Define cover crop intensity with NDVI thresholds

Using the maximum NDVI composite derived in the previous cell, several bins of varied vegetation cover are defined using NDVI thresholds.

Following the guideline in [this paper](https://www.jswconline.org/content/jswc/70/6/340.full.pdf), we define Minimal Vegetation Cover as anything with NDVI < 0.29, Low Vegetation Cover with 0.29 <= NDVI < 0.40, Medium Vegetation Cover with 0.40 <= NDVI < 0.53, and High Vegetation Cover with NDVI >= 0.53.

Each of these vegetation cover bins are calculated separately below, then displayed on the interactive map as individual layers. The Minimal, Low, Medium, and High Cover classes display on the map as red, orange, yellow, and green, respectively.

In [9]:
# Minimal
min_veg = s2_max_ndvi < 0.29
min_veg.mask(~combined).mask(~min_veg).visualize(
    'Minimal Cover Crop',
    colormap='bwr',
    checkerboard=False,
    year=cc_m.year_select)

# Low
low_veg = (s2_max_ndvi >= 0.29) & (s2_max_ndvi < 0.40)
low_veg.mask(~combined).mask(~low_veg).visualize(
    'Low Cover Crop',
    colormap='Wistia',
    checkerboard=False,
    year=cc_m.year_select)

# Medium
med_veg = (s2_max_ndvi >= 0.40) & (s2_max_ndvi < 0.53)
med_veg.mask(~combined).mask(~med_veg).visualize(
    'Medium Cover Crop',
    colormap='autumn',
    checkerboard=False,
    year=cc_m.year_select)

# High
high_veg = (s2_max_ndvi >= 0.53)
high_veg.mask(~combined).mask(~high_veg).visualize(
    'High Cover Crop',
    colormap='RdYlGn',
    checkerboard=False,
    year=cc_m.year_select)